In [61]:
import os;import sys
import pandas as pd;import numpy as np;
import networkx as nx
import numpy.random as npr 
from scipy.stats.distributions import chi2
from scipy.special import gamma, factorial

## GRAPH FUNCTIONS

In [62]:
def makeDiGraph(log_df):
    """A function to build a bipartite graph from an action log.
    - log_df (dataframe) contains timestamped network links (Source | Target | Timestamp) """
    
    B = nx.DiGraph()
    B.add_nodes_from(log_df['Source'].unique(), bipartite=0)
    B.add_nodes_from(log_df['Target'].unique(), bipartite=1)
    
    for i,row in log_df.iterrows():
        e, year = (row['Source'],row['Target']), row['Timestamp']
        if e in B.edges():
            w = B.edges[e]['weight']
            B.add_edge(*e,weight=w+1)
        else:
            B.add_edge(*e,weight=1)
    return B

In [63]:
def addWeightedEdge(B,e):
    b = B.copy()
    if e in b.edges():
        w = b.edges[e]['weight']
        b.add_edge(*e,weight=w+1)
    else:
        b.add_edge(*e,weight=1)
    return b

## MODEL SIMULATION FUNCTIONS

In [64]:
def alias_setup(probs):
    """
    A function to efficiently sample a non-uniform probability distribution using Vose's method:
    https://lips.cs.princeton.edu/the-alias-method-efficient-sampling-with-many-discrete-outcomes/
    
    """
    K       = len(probs)
    q       = np.zeros(K)
    J       = np.zeros(K, dtype=int)

    smaller = []  # Sort the data into the outcomes with probabilities
    larger  = []  # that are larger and smaller than 1/K.
    for kk, prob in enumerate(probs):
        q[kk] = K*prob
        if q[kk] < 1.0:
            smaller.append(kk)
        else:
            larger.append(kk)
            
    while len(smaller) > 0 and len(larger) > 0: 
        small = smaller.pop() 
        large = larger.pop() 

        J[small] = large
        q[large] = q[large] - (1.0 - q[small])

        if q[large] < 1.0:
            smaller.append(large)
        else:
            larger.append(large)

    return J, q

def alias_draw(J, q):
    K  = len(J)

    # Draw from the overall uniform mixture.
    kk = int(np.floor(npr.rand()*K))

    # Draw from the binary mixture, either keeping the
    # small one, or choosing the associated larger one.
    if npr.rand() < q[kk]:
        return kk
    else:
        return J[kk]

In [1]:
def simGraph(b, log_df, source_nodes, target_nodes, feature_df, features): 
    """ A function to simulate the 'fixed-effects' network model
    wherein the probability of an attachment to a target node is proportional to an exponentiated 
    utility function: p ~ exp(u_nt) where u = b * f_nt for feature f and node n at time t
     - b (list) is the vector of utility parameters 
     - log (dataframe) contains the timestamped network links ( Source | Target | Timestamp )
     - target_nodes (list)
     - source_nodes (list)
     - feature_df(dataframe) contains the feature variables to be used in the utility function...
                                                      ( Country | Year | Feature 1 | ... | Feature N )
     - features (list) is a vector of features                                                
     Returns the final network B and a dictionary of edges keyed by year.
    """
        
    # Initialize the bipartite graph with source nodes and target nodes
    B = nx.DiGraph()
    B.add_nodes_from(source_nodes,bipartite=0)
    B.add_nodes_from(target_nodes,bipartite=1)
    
    # Initalize a list of edges keyed by year
    edges = {y:[] for y in sorted(log_df['Timestamp'].unique())}
    
    # Create dictionary from feature dataframe indexing the value of feature f by target node n, and year t
    f_nt = {f:{n:{row['Year']:row[f] for j, row in feature_df.groupby('Country').get_group(n).iterrows()}\
                 for n in target_nodes} for f in features}
    
    for i, row in log_df.iterrows():
        t, source = row['Timestamp'], row['Source']
       
        # Create dictionary to contain probability of attachment to each target node n at time t
        p = {}
        for n in target_nodes:
            p[n] = np.exp(np.dot([f_nt[f][n][t] for f in features], b))
            
        
        Z = sum([p[n] for n in target_nodes])
        P = [p[n]/Z if Z > 0 else 0 for n in target_nodes] # create normalized list of probabilities
        
        # Draw the target node from the probability distribution P
        J,q = alias_setup(P)
        target = target_nodes[alias_draw(J, q)]
    
        # Add the new edge to the graph
        edges[t].append((source,target))
        B = addWeightedEdge(B,(source,target))
    
    return B, edges

# MODEL ESTIMATION FUNCTIONS

In [66]:
def logL(b, log_df, target_nodes, feature_df, features):
    """ A function to calucate the loglikelhood of the dataset based on the 'fixed-effects' network model
    wherein the probability of an investment is proportional to an exponentiated utility function:
    p ~ exp(u_nt) where u = b * f_nt for feature f and country n at time t
     - b (list) is parameter vector to be estimated, 
     - log_df (dataframe) containing the timestamped network links ( Source | Target | Timestamp )
     - target_nodes (list)
     - feature_df (dataframe) contains feature variables to be tested in the utility function...
                                                      ( Country | Year | Feature 1 | ... | Feature N )"""
    
    
    # Initialize likelihood function L and regularization function S
    L = 0
    
    # Create dictionary from feature dataframe indexing the value of feature f by country n, and year t
    f_nt = {f:{n:{row['Year']:row[f] for j, row in feature_df.groupby('Country').get_group(n).iterrows()}\
                 for n in target_nodes} for f in features}
    
    # Calculate likelihood for each network link
    for i, row in log_df.iterrows():
        source, target, t = row['Source'], row['Target'], row['Timestamp']
        
        u = {} # Initialize dictionary to store utility of each country u = b * f_nt
        for i,n in enumerate(target_nodes):
            u[n] = np.dot([f_nt[f][n][t] for f in features], b)

        L += u[target] - np.log(sum([np.exp(u[n]) for n in target_nodes])) 
                
    return L 

# MODEL EVALUATION FUNCTIONS

In [67]:
def X2_p(logL0, logLM, k):
    """ A function to compute the Xi-squared test statistic for the Wald's likelihood ratio test
    - logL0 is the loglikelihood of the null model 
    - logLM is the loglikelihood of the model to be evalulated
    - k is the number of degrees of freedom
    """
    X2 = -2*(logL0-logLM)
    return chi2.sf(X2,k)